# Questions and Answers using Llama 2

Model: Llama 2 7b-chat

Data: Paul Grahama test data set

In [1]:
from reginald.models.setup_llm import setup_llm

import pandas as pd

from reginald.models.models.llama_index import (
    setup_settings,
    LlamaIndexLlamaCPP,
    set_global_tokenizer,
    compute_default_chunk_size,
)

/Users/kgoldmann/Documents/Projects/reginald/.venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
from llama_index.core.evaluation import DatasetGenerator, RelevancyEvaluator
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Response
from llama_index.llms.openai import OpenAI
from llama_index.llms.ollama import Ollama

from llama_index.core import (
    StorageContext,
    load_index_from_storage,
)

from transformers import AutoTokenizer
import nest_asyncio

nest_asyncio.apply()

In [3]:
# Use ollama
ollama_llm = Ollama(model="llama2:7b-chat", request_timeout=60.0)

In [4]:
chunk_size = compute_default_chunk_size(
    max_input_size=4096, k=3
)  # calculate chunk size

In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf"
).encode  # load tokenizer
set_global_tokenizer(tokenizer)

In [6]:
settings = setup_settings(
    llm=ollama_llm,
    max_input_size=4096,
    num_output=512,
    chunk_overlap_ratio=0.1,
    chunk_size=chunk_size,
    k=3,
    tokenizer=tokenizer,
)  # these are settings for the storage context

## Load in the Data

In [7]:
# To load in the handbook

# storage_context = StorageContext.from_defaults(
#     persist_dir="../../data/llama_index_indices/handbook/"
# )

# vector_index = load_index_from_storage(
#     storage_context=storage_context,
#     settings=settings,
# )  # load the data index from storage

In [8]:
# To load in the example Paul Graham data
reader = SimpleDirectoryReader("./data/paul_graham/")
documents = reader.load_data()
vector_index = VectorStoreIndex.from_documents(documents)

In [9]:
dataset_generator = DatasetGenerator.from_documents(
    vector_index.docstore.docs.values(),
    num_questions_per_chunk=1,
    show_progress=True,
    llm=ollama_llm
)

Parsing nodes:   0%|          | 0/26 [00:00<?, ?it/s]

/Users/kgoldmann/Documents/Projects/reginald/.venv/lib/python3.11/site-packages/llama_index/core/evaluation/dataset_generation.py:213: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


## Generate Questions

In [10]:
eval_questions = dataset_generator.generate_questions_from_nodes(
    5
)  # generate questions from the documents

100%|██████████| 5/5 [01:12<00:00, 14.50s/it]
/Users/kgoldmann/Documents/Projects/reginald/.venv/lib/python3.11/site-packages/llama_index/core/evaluation/dataset_generation.py:310: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [11]:
eval_questions

['Of course! Here are 5 questions based on the given context:',
 "What was Paul Graham's main focus outside of school before college?",
 'What was the language used in the IBM 1401 that Paul and his friend Rich Draves used?',
 "Can you describe Paul's experience with the Heathkit microcomputer he built himself?",
 'Why did Paul convince his father to buy a TRS-80 computer for him?']

## Answer the Questions

In [12]:
evaluator_gpt4 = RelevancyEvaluator(llm=ollama_llm)

In [14]:
# for every value in eval_questions, get the response from the llm
all_dfs = []

for i in range(len(eval_questions)):
    query_engine = vector_index.as_query_engine()
    response_vector = query_engine.query(eval_questions[i])
    eval_result = evaluator_gpt4.evaluate_response(
        query=eval_questions[i], response=response_vector
    )

    # append to the all_dfs list
    all_dfs.append({'eval_result': eval_result,
                    'question': eval_questions[i],
                    'response': response_vector})


In [20]:
# extract the different elements
questions = [x['question'] for x in all_dfs]
response = [x['response'] for x in all_dfs]
sources = [x['response'].source_nodes for x in all_dfs]
sources = [x[0].node.get_content() for x in sources]
match = [x['eval_result'].passing for x in all_dfs] # does the response match the source

In [21]:
# extract the questions
combined_df = pd.DataFrame({'questions': questions, 'response': response, 'sources': sources, 'match': match})
combined_df.to_csv("data/paul_QandA.csv", index=False)

In [24]:
combined_df_out = combined_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        }
    )
display(combined_df_out)

,questions,response,sources,match
0,Of course! Here are 5 questions based on the given context:,"Of course! Here are 5 questions based on the given context: 1. What were some of the key topics discussed during Paul Graham's visit to Rich Draves at CMU in 1988? 2. How did Rich Draves' work on slow systems inform Paul Graham's approach to building systems that would last and be relevant in the face of rapidly changing technology? 3. Can you provide an example of how Paul Graham may have been influenced by Rich Draves' expertise in computer science during his visit? 4. What was the main goal of Paul Graham's visit to CMU in 1988, and how did it relate to his broader interests in software development? 5. How might Paul Graham's collaboration with Rich Draves have contributed to the development of new approaches to building systems that would last and be relevant in the face of rapidly changing technology?","Anyone who wanted one to play around with could have one. I was briefly tempted, but they were so slow by present standards; what was the point? No one else wanted one either, so off they went. That was what happened to systems work. I wanted not just to build things, but to build things that would last. In this dissatisfied state I went in 1988 to visit Rich Draves at CMU, where he was in grad school. One day I went to visit the Carnegie Institute, where I'd spent a lot of time as a kid. While looking at a painting there I realized something that might seem obvious, but was a big surprise to me. There, right on the wall, was something you could make that would last. Paintings didn't become obsolete. Some of the best ones were hundreds of years old. And moreover this was something you could make a living doing. Not as easily as you could by writing software, of course, but I thought if you were really industrious and lived really cheaply, it had to be possible to make enough to survive. And as an artist you could be truly independent. You wouldn't have a boss, or even need to get research funding. I had always liked looking at paintings. Could I make them? I had no idea. I'd never imagined it was even possible. I knew intellectually that people made art — that it didn't just appear spontaneously — but it was as if the people who made it were a different species. They either lived long ago or were mysterious geniuses doing strange things in profiles in Life magazine. The idea of actually being able to make art, to put that verb before that noun, seemed almost miraculous. That fall I started taking art classes at Harvard. Grad students could take classes in any department, and my advisor, Tom Cheatham, was very easy going. If he even knew about the strange classes I was taking, he never said anything. So now I was in a PhD program in computer science, yet planning to be an artist, yet also genuinely in love with Lisp hacking and working away at On Lisp. In other words, like many a grad student, I was working energetically on multiple projects that were not my thesis. I didn't see a way out of this situation. I didn't want to drop out of grad school, but how else was I going to get out? I remember when my friend Robert Morris got kicked out of Cornell for writing the internet worm of 1988, I was envious that he'd found such a spectacular way to get out of grad school. Then one day in April 1990 a crack appeared in the wall. I ran into professor Cheatham and he asked if I was far enough along to graduate that June. I didn't have a word of my dissertation written, but in what must have been the quickest bit of thinking in my life, I decided to take a shot at writing one in the 5 weeks or so that remained before the deadline, reusing parts of On Lisp where I could, and I was able to respond, with no perceptible delay ""Yes, I think so. I'll give you something to read in a few days."" I picked applications of continuations as the topic. In retrospect I should have written about macros and embedded languages. There's a whole world ther

In [19]:
display_eval_df(eval_questions[1], response_vector, eval_result)

,Query,Response,Source,Evaluation Result
0,What was Paul Graham's main focus outside of school before college?,"Rewrite: Paul convinced his father to buy a TRS-80 computer for him because he was eager to explore the revolutionary technology and its boundless potential, much like how he longed to create lasting structures during his visit to Rich Draves at CMU in 1988.","What I Worked On February 2021 Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep. The first programs I tried writing were on the IBM 1401 that our school district used for what was then called ""data processing."" This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights. The language we used was an early version of Fortran. You had to type programs on punch cards, then stack them in the card reader and press a button to load the program into memory and run it. The result would ordinarily be to print something on the spectacularly loud printer. I was puzzled by the 1401. I couldn't figure out what to do with it. And in retrospect there's not much I could have done with it. The only form of input to programs was data stored on punched cards, and I didn't have any data stored on punched cards. The only other option was to do things that didn't rely on any input, like calculate approximations of pi, but I didn't know enough math to do anything interesting of that type. So I'm not surprised I can't remember any programs I wrote, because they can't have done much. My clearest memory is of the moment I learned it was possible for programs not to terminate, when one of mine didn't. On a machine without time-sharing, this was a social as well as a technical error, as the data center manager's expression made clear. With microcomputers, everything changed. Now you could have a computer sitting right in front of you, on a desk, that could respond to your keystrokes as it was running instead of just churning through a stack of punch cards and then stopping. [1] The first of my friends to get a microcomputer built it himself. It was sold as a kit by Heathkit. I remember vividly how impressed and envious I felt watching him sitting in front of it, typing programs right into the computer. Computers were expensive in those days and it took me years of nagging before I convinced my father to buy one, a TRS-80, in about 1980. The gold standard then was the Apple II, but a TRS-80 was good enough. This was when I really started programming. I wrote simple games, a program to predict how high my model rockets would fly, and a word processor that my father used to write at least one book. There was only room in memory for about 2 pages of text, so he'd write 2 pages at a time and then print them out, but it was a lot better than a typewriter. Though I liked programming, I didn't plan to study it in college. In college I was going to study philosophy, which sounded much more powerful. It seemed, to my naive high school self, to be the study of the ultimate truths, compared to which the things studied in other fields would be mere domain knowledge. What I discovered when I got to college was that the other fields took up so much of the space of ideas that there wasn't much left for these supposed ultimate truths. All that seemed left for philosophy were edge cases that people in other fields felt could safely be ignored. I couldn't have put this into words when I was 18.",True
